In [18]:
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import LineString
from pyproj import Transformer

In [15]:
data_dir = '../'
routes_gdf = gpd.read_file(data_dir + 'MBTA_Systemwide_GTFS_Map/MBTA_Systemwide_GTFS_Map.shp')
neighborhoods_gdf = gpd.read_file(data_dir + '/Census2020_BG_Neighborhoods/Census2020_BG_Neighborhoods.shp')
routes_gdf, neighborhoods_gdf

(     OBJECTID shape_id route_id route_shor  \
 0       66654   010111        1          1   
 1       66655   010118        1          1   
 2       66656   040070        4          4   
 3       66657   040071        4          4   
 4       66658   040072        4          4   
 ..        ...      ...      ...        ...   
 674     67379   970075       97         97   
 675     67413   990123       99         99   
 676     67414   990124       99         99   
 677     67415   990125       99         99   
 678     67416   990126       99         99   
 
                                      route_long    route_desc route_type  \
 0               Harvard Square - Nubian Station       Key Bus          3   
 1               Harvard Square - Nubian Station       Key Bus          3   
 2                   North Station - Tide Street  Commuter Bus          3   
 3                   North Station - Tide Street  Commuter Bus          3   
 4                   North Station - Tide Street 

In [9]:
routes_by_neighborhood = pd.DataFrame()

In [16]:
neighborhoods_gdf['BlockGr202'].unique()


In [45]:
# transformer = Transformer.from_crs(routes_gdf.crs, neighborhoods_gdf.crs)
# new_route = transformer.transform(first_route)

routes_gdf = routes_gdf.to_crs(neighborhoods_gdf.crs)
first_route = routes_gdf.iloc[0]

df = pd.DataFrame(columns=['route_id', 'neighborhood'])

# Loop through each geometry in route_gdf
for idx, route in routes_gdf.iterrows():
    # Check if the route intersects with any geometry in neighborhood_gdf
    intersection = neighborhoods_gdf.geometry.intersects(route.geometry)
    # If there is an intersection, add the 'route_id' and 'neighbor' to the DataFrame
    if intersection.any():
        intersecting_neighbors = neighborhoods_gdf[intersection]
        for _, neighbor in intersecting_neighbors.iterrows():
            new_df = pd.DataFrame([[route.route_id, neighbor.BlockGr202]],
                   columns=['route_id', 'neighborhood'])
            df = pd.concat([new_df, df], ignore_index=True)

96

In [55]:
test_df = df.drop_duplicates()

In [60]:
test_df.to_csv(data_dir + 'routes_and_neighborhoods.csv', index=False)